# EDA for EIA Data

## Load in libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
color_pal = sns.color_palette()
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import pickle

SyntaxError: invalid syntax (3174205862.py, line 10)

In [ ]:
gen_df = pd.read_parquet('../data/energy_demand.parquet')
weather_df = pd.read_parquet('../data/weather.parquet')
XLE_df = pd.read_csv('../data/XLE.csv')
holidays = pd.read_csv('../data/holiday.csv')

In [ ]:
gen_df['respondent-name'] = gen_df['respondent-name'].replace('Florida', 'Florida Power and Light')

In [ ]:
gen_df['period'] = pd.to_datetime(gen_df.period)

# Create a new column to merge weather data on
- merge on period and name

In [ ]:
gen_df['name'] = gen_df['respondent-name']
gen_df['name'].value_counts()

In [ ]:
respondents =['Electric Reliability Council of Texas, Inc.' ,
              'New York Independent System Operator',
              'Florida Power and Light' ,
               'California Independent System Operator',]

In [ ]:
gen_df['name'] = gen_df['name'].replace(respondents[3], 'Los Angeles')
gen_df['name'] = gen_df['name'].replace(respondents[2], 'Tampa')
gen_df['name'] = gen_df['name'].replace(respondents[0], 'Austin')
gen_df['name'] = gen_df['name'].replace(respondents[1], 'new york city')

In [ ]:
energy_df = pd.merge(gen_df,weather_df, how='inner', on=['name','period'],suffixes=('','2'))

In [ ]:
XLE_df = XLE_df.set_index('Date')
XLE_df.index = pd.to_datetime(XLE_df.index)
XLE_df = XLE_df[['Adj Close']].copy()
XLE_df['year'] = XLE_df.index.year
XLE_df['dayofyear'] = XLE_df.index.dayofyear

In [ ]:
XLE_df.sort_index(inplace=True)

In [ ]:
XLE_df['Adj_Close_30D_lag'] = XLE_df['Adj Close'].shift(freq='28D')
XLE_df['Adj_Close_60D_lag'] = XLE_df['Adj Close'].shift(freq='56D')
energy_df = pd.merge(energy_df,XLE_df,how='left', on=['year','dayofyear'])

In [ ]:
holidays =holidays.drop('Date',axis=1)
energy_df = pd.merge(energy_df,holidays, how='left', on=['year','dayofyear'])

# Check for nulls
- use pandas bfill to fill in missing data
- change period to datetime
- filer data
- change column names

In [ ]:
energy_df['value'] = energy_df['value'].bfill()
energy_df['solarradiation'] = energy_df['solarradiation'].bfill()
energy_df['uvindex'] = energy_df['uvindex'].bfill()
energy_df['snow'] = energy_df['snow'].bfill()
energy_df['Adj Close'] = energy_df['Adj Close'].bfill()
energy_df['Adj_Close_30D_lag'] = energy_df['Adj Close'].bfill()
energy_df['Adj_Close_60D_lag'] = energy_df['Adj Close'].bfill()

In [ ]:
energy_df = energy_df.set_index('period')
energy_df.index = pd.to_datetime(energy_df.index)

In [ ]:
energy_df["Is Weekend"] = energy_df.index.day_name().isin(['Saturday', 'Sunday'])

In [ ]:
energy_df.columns;

In [ ]:
columns = ['respondent-name','value',
        'year', 'month', 'days', 'hour', 'quarter', 'dayofyear',
        'temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'snow', 'snowdepth', 'windspeed', 'winddir', 'sealevelpressure',
       'cloudcover', 'visibility', 'solarradiation', 'uvindex', 'conditions',
       'icon','Adj Close', 'Adj_Close_30D_lag', 'Adj_Close_60D_lag', 'Holiday','Is Weekend']

In [ ]:
energy_df = energy_df[columns].copy()

In [ ]:
energy_df.rename(columns={'respondent-name':'respondent',}, inplace=True)
energy_df = pd.get_dummies(energy_df, columns=['conditions','icon','Holiday','Is Weekend'], drop_first=True, dtype=float)

In [ ]:
for n,res in enumerate(respondents):
    ax = plt.subplot(2,2,n+1)
    energy_df.query(f"respondent == '{res}' ")['value'].plot(kind='hist', bins =500,ax =ax, figsize=(18,8))
    ax.set_title(label =f'{res}',fontsize =12)
    plt.tight_layout()

In [ ]:
for n,res in enumerate(respondents):
    ax = plt.subplot(2,2,n+1)
    energy_df.query(f"respondent == '{res}' ")['value'].plot(ax =ax, figsize=(16,9))
    ax.set_title(label =f'{res}',fontsize =14)
    plt.tight_layout()

In [ ]:
NY = energy_df.query("respondent== 'New York Independent System Operator' and value > 11000 and value <32000").copy()
FL = energy_df.query("respondent == 'Florida Power and Light' and value >14000 and value < 49000").copy()
CA = energy_df.query("respondent ==  'California Independent System Operator' and value >15000 and value <46000").copy()
TX = energy_df.query("respondent ==  'Electric Reliability Council of Texas, Inc.' and value <75000 ").copy()

In [ ]:
NY['demand_3d_lag'] = NY['value'].shift(72)
FL['demand_3d_lag'] = FL['value'].shift(72)
CA['demand_3d_lag'] = CA['value'].shift(72)
TX['demand_3d_lag'] = TX['value'].shift(72)

In [ ]:
NY.dropna(inplace=True)
FL.dropna(inplace=True)
CA.dropna(inplace=True)
TX.dropna(inplace=True)

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(20,12))
TX['value'].plot(ax=ax[0, 0], title =respondents[0])
NY['value'].plot(ax=ax[0, 1], title = respondents[1])
FL['value'].plot(ax=ax[1, 0], title =respondents[2])
CA['value'].plot(ax=ax[1, 1], title =respondents[3])
fig.suptitle("Megawatts generated", fontsize=20)
plt.tight_layout(pad=1)
plt.show()

In [ ]:
def create_box_plot(frequency= 'month'):
    frequency= frequency.lower()
    fig, ax = plt.subplots(2,2 ,figsize=(20,12))
    sns.boxplot(data=TX, x=frequency, y='value', palette='Blues', ax=ax[0, 0], width=.4, linewidth=.8, fliersize=4)
    ax[0,0].set_title(respondents[0])
    sns.boxplot(data=NY, x=frequency, y='value', palette='Blues', ax=ax[0, 1], width=.4, linewidth=.8, fliersize=4)
    ax[0,1].set_title(respondents[1])
    sns.boxplot(data=FL, x=frequency, y='value', palette='Blues', ax=ax[1, 0], width=.4, linewidth=.8, fliersize=4)
    ax[1,0].set_title(respondents[2])
    sns.boxplot(data=CA, x=frequency, y='value', palette='Blues', ax=ax[1, 1], width=.4, linewidth=.8, fliersize=4)
    ax[1,1].set_title(respondents[3])
    plt.tight_layout(pad=2)
    plt.show()

In [ ]:
# hour = create_box_plot('hour')
# month = create_box_plot('month')
# quarter = create_box_plot('quarter')

In [ ]:
# NY.to_parquet('../data/ML_Data/NY', engine='pyarrow')
# FL.to_parquet('../data/ML_Data/FL', engine='pyarrow')
# CA.to_parquet('../data/ML_Data/CA', engine='pyarrow')
# TX.to_parquet('../data/ML_Data/TX', engine='pyarrow')

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,10), sharex=True)
tss = TimeSeriesSplit(n_splits=3, test_size=24*547, gap=24)
fold = 0
for train_idx, val_idx in tss.split(NY):
    train = NY.iloc[train_idx]
    test = NY.iloc[val_idx]
    train['value'].plot(ax=axs[fold],
                          label='Training Set',
                          title=f'Data Train/Test Split Fold {fold}')
    test['value'].plot(ax=axs[fold],
                         label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')
    fold += 1
plt.show()

In [ ]:
TARGET = 'value'
FEATURES = NY.columns[2:]

In [ ]:
TX_params1 = {
    "base_score": 0.5,
    "booster": "gbtree",
    "n_estimators": 8000,
    "early_stopping_rounds": 100,
    "max_depth": 12,
    "learning_rate": 0.1,
    "subsample": .9,
    "reg_lambda": 20,
    "reg_alpha": 20,
    "min_split_loss": 20,
    "min_child_weight": 15,
    "colsample_bytree": .8,
    "colsample_bylevel": .8,
    "colsample_bynode": 1,
    "random_state": 0,
    "eval_metric": ['rmse','mae']
}

In [ ]:
def xgb_train(df, params=None, week_list=None, show_graph = "no"):
    if params is None:
        params = {}
    tss = TimeSeriesSplit(n_splits=3, test_size=24*547, gap=24)
    df = df.sort_index()
    fold = 0
    preds = []
    scores_rmse = []
    scores_mae = []
    for train_idx, val_idx in tss.split(df):
        train = df.iloc[train_idx]
        test = df.iloc[val_idx]
        X_train = train[FEATURES]
        y_train = train[TARGET]
        X_test = test[FEATURES]
        y_test = test[TARGET]
        reg = xgb.XGBRegressor()
        reg.set_params(**params)
        reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=1500,)
        y_pred = reg.predict(X_test)
        preds.append(y_pred)
        score_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        scores_rmse.append(score_rmse)
        scores_mae.append(mean_absolute_error(y_test,y_pred))
    print(f'Score across folds {np.mean(scores_rmse):0.4f}')
    print(f'Fold scores:{scores_rmse}')
    print(f'Score across folds {np.mean(scores_mae):0.4f}')
    print(f'Fold scores:{scores_mae}')

    if show_graph.lower() == 'yes':
        if week_list is None:
            week = ['03-20-2022', '03-27-2022']
        else:
            week = week_list
        test['prediction'] = reg.predict(X_test)
        df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
        ax = df[['value']].plot(figsize=(15, 6))
        df['prediction'].plot(ax=ax, style='.')
        plt.legend(['Actual', 'Predicted'],loc='upper right')
        ax.set_title('texas Actual VS. Predicted')
        plt.show()


        ax = df.loc[(df.index > week[0]) & (df.index < week[1])]['value'] \
        .plot(figsize=(15, 6), title='Week Of Data')
        df.loc[(df.index > week[0]) & (df.index < week[1])]['prediction'] \
            .plot()
        plt.legend(['Actual','Predicted'],loc='upper right')
        plt.show()

        fig, ax = plt.subplots(figsize=(10,10))
        xgb.plot_importance(reg,importance_type='gain', ax=ax, show_values=False, max_num_features=30)
    return reg,df

In [ ]:
model_TX, TX_df =xgb_train(TX, TX_params1, show_graph ='yes')

In [ ]:
model_NY, NY_df =xgb_train(NY, TX_params1, show_graph='no')

In [ ]:
model_CA, CA_df =xgb_train(CA, TX_params1, show_graph='no')

In [ ]:
model_FL, FL_df =xgb_train(FL, TX_params1, show_graph='no')

In [ ]:
filename = 'Texas_model.pkl'
pickle.dump(model_TX,open(filename,'wb'))

In [ ]:
ax = TX_df[['value']].plot(figsize=(15, 6))
TX_df['prediction'].plot(ax=ax, style='.')
plt.legend(['Actual', 'Predicted'],loc='upper right')
ax.set_title('texas Actual VS. Predicted')
plt.show()

In [ ]:
m = pickle.load(open(filename, 'rb'))

In [ ]:
import plotly.express as px

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(m,importance_type='gain', ax=ax, show_values=False, max_num_features=30)

In [ ]:
fi = pd.DataFrame(data=m.feature_importances_,
   index = m.get_booster().feature_names,
    columns=['importance'])


In [ ]:
fi.sort_values()

In [ ]:
x = m.get_booster()

In [ ]:
x.feature_importances_

In [ ]:
# test['error'] = np.abs(test[TARGET] - test['prediction'])
# test['date'] = test.index.date
# test.groupby(['date'])['error'].mean().sort_values(ascending=False).head(10)

In [ ]:
# def create_importance(model):
#     fig, ax = plt.subplots(figsize=(10,10))
#     xgb.plot_importance(model,importance_type='weight', ax=ax, show_values=False, max_num_features=30)

In [ ]:
# reg.save_model('model.json')

In [ ]:
# def create_graphs(week=None):
#     if week is None:
#         week = ['12-20-2022', '12-27-2022']
#     test['prediction'] = reg.predict(X_test)
#     df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
#     ax = df[['value']].plot(figsize=(15, 6))
#     df['prediction'].plot(ax=ax, style='.')
#     plt.legend(['Actual', 'Predicted'],loc='upper right')
#     ax.set_title('Actual VS. Predicted')
#     plt.show()
#
#
#     ax = df.loc[(df.index > week[0]) & (df.index < week[1])]['value'] \
#     .plot(figsize=(15, 6), title='Week Of Data')
#     df.loc[(df.index > week[0]) & (df.index < week[1])]['prediction'] \
#         .plot()
#     plt.legend(['Actual','Predicted'],loc='upper right')
#     plt.show()